# Modeling notebook

In [ ]:
import os
from src.train import train_main
os.environ['DATASET_PATH'] = os.environ.get('DATASET_PATH', '')
train_main(['--data', os.environ['DATASET_PATH'], '--task', 'mortality', '--quick'])